In [2]:
import os
import ray
import json
import tqdm
import torch
import logging
import numpy as np
from transformers import AutoTokenizer
from typing import List

from full_doc_loader import FullDocLoader

logger = logging.getLogger(__name__)

In [3]:
ray.init()

2020-03-03 13:26:36,100	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-03 13:26:36,101	INFO resource_spec.py:216 -- Starting Ray with 23.58 GiB memory available for workers and up to 11.81 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '169.237.10.101',
 'redis_address': '169.237.10.101:18481',
 'object_store_address': '/tmp/ray/session_2020-03-03_13-26-36_099919_14072/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-03_13-26-36_099919_14072/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-03-03_13-26-36_099919_14072'}

In [4]:
class FullDocIterator:
    def __init__(self, processed_docs, sep_token: str, max_seq_length:int, allow_cross_doc=True):
        self.processed_docs = processed_docs
        self.max_seq_length = max_seq_length
        self.allow_cross_doc = allow_cross_doc
        self.total_num_docs = len(processed_docs)
        self.sep_token = sep_token

    def __iter__ (self):
        # shuffle the indices
        indices = np.arange(self.total_num_docs)
        np.random.shuffle(indices)
        current_seq = []
        history_pointer = 0

        for doc_index in indices:
            # randomly sample a document
            doc = self.processed_docs[doc_index]
            
            if not self.allow_cross_doc:
                for i in range(0, len(doc), self.max_seq_length):
                    yield doc[i:i+self.max_seq_length], i==0
            else:
                while history_pointer < len(doc):
                    # history pointer for the current document
                    next_pointer = history_pointer + self.max_seq_length - len(current_seq)
                    doc_seg = doc[history_pointer:next_pointer]
                    current_seq.extend(doc_seg)
                    
                    if_start_doc = history_pointer == 0
                    history_pointer = history_pointer + len(doc_seg)
                    # print(history_pointer)
                    
                    if len(current_seq) == self.max_seq_length:
                        yield current_seq, if_start_doc
                        current_seq = []
                    
                # if the document is over
                history_pointer = 0
                if len(current_seq) > 0:
                    current_seq = current_seq + [self.sep_token]

class FullDocBatchIterator:
    def __init__(self, tokenizer, corpus_path:str, batch_size:int, max_seq_length:int, rank:int = 0):
        """
        Args:
            corpus_path: directory path to store the corpus sectors
            rank: for distributed learning.
        """ 
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_seq_length = max_seq_length
        self.total_num_sectors = len(os.listdir(corpus_path))
        self.current_sector_id = rank
        self.next_sector_id = self.get_next_sector_id(self.current_sector_id)
        
        self.corpus_loader = FullDocLoader(tokenizer, corpus_path=corpus_path)
        
        # process the data and save it into cache
        self.ray_cache_docs = None
    
    def get_next_sector_id(self, current_sector_id):
        return (current_sector_id + 1) % self.total_num_sectors
    
    def __iter__(self):
        iterators = self.create_corpus_iterators(self.current_sector_id)
        
        while True:
            try:
                # TODO: extend it with Ray
                batch = [next(iterators[i]) for i in range(self.batch_size)]
                yield batch
                
            except StopIteration:
                # after the iterator finishes, load the next sector
                # update self.current_sector_id
                self.current_sector_id = self.next_sector_id
                self.next_sector_id = self.get_next_sector_id(self.current_sector_id)
                iterators = self.create_corpus_iterators(self.current_sector_id)
                
    def create_corpus_iterators(self, corpus_sector_id):
        if self.ray_cache_docs is None:
            # if it is the beginging
            processed_docs = self.corpus_loader.load_sector(self.current_sector_id)
        else:
            # if ray_cache is already set 
            processed_docs = ray.get(self.ray_cache_docs)
        
        iterators = [iter(FullDocIterator(processed_docs, 
                                          sep_token=self.tokenizer.sep_token, 
                                          max_seq_length=self.max_seq_length)) 
                     for i in range(self.batch_size)]
        
        # set ray cache
        self.ray_cache_docs = _ray_load_sector.remote(self.corpus_loader, self.next_sector_id)
            
        return iterators

@ray.remote
def _ray_load_sector(corpus_loader, sector_id):
    return corpus_loader.load_sector(sector_id)

In [5]:
list(range(0, 10-5, 5))

[0]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [7]:
corpus_iter = FullDocBatchIterator(tokenizer, 
                                   corpus_path="/data/en-corpus/SECTOR/",
                                   batch_size=1,
                                   max_seq_length=128, 
                                   rank=0)

In [12]:
iterator = iter(corpus_iter)

In [13]:
for i in tqdm.tqdm(iterator):
    if corpus_iter.current_sector_id == (corpus_iter.total_num_sectors - 1):
        break

15618046it [09:11, 252756.67it/s]

(pid=14222) Processing Data. Takes about 10 mins


15671252it [09:11, 257427.02it/s]

 19%|█▉        | 2532/13470 [00:12<00:58, 186.70it/s]


15671252it [09:25, 257427.02it/s]

...

46478628it [1:29:46, 252591.59it/s]

 44%|████▍     | 5911/13470 [00:30<00:31, 239.70it/s]


 74%|███████▍  | 10023/13470 [00:48<00:14, 230.40it/s]


100%|██████████| 13470/13470 [01:01<00:00, 220.11it/s]
(pid=14222) Saving Into Cache


KeyboardInterrupt: 

64

In [1]:
!ray stop